# Data prep


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
pd.set_option('display.max_columns', None)

In [ ]:
#Read in old AWI
gdf_old = gpd.read_file("../Old_AWI_Wilts/Old_AWI_Wilts.shp")
# Updated AWI
gdf_new = gpd.read_file("../New_AWI_Wilts/AWI_sources_updated_5_resolved.shp")
gdf_new_reduced = gdf_new[['NAME_OS', 'NAME_EP1', 'P3_UID', 'P3_IGNORE', 'C21_AP', 'EPOCH_2',
       'EPOCH_1', 'C20_AP', 'C20_AP_REF', 'TITHE_REF', 'C19_EM', 'C19_EM_REF',
       'OSD', 'OSD_REF', 'CM_1773', 'C18_EM', 'C18_EM_REF', 'C17_EM',
       'C17_EM_REF', 'EM', 'EM_REF', 'OTHER', 'OTHER_REF', 'NAME_TITHE',
       'NAME_OSD', 'NAME_EM', 'TITHE_REF_', 'TITHE_LAND', 'TITHE_PLOT',
       'TITHE_PL_1', 'COMMENTS', 'url', 'OSNAMES_UR', 'CHECK_NAME',
       'TO_CHECK_N', 'NOTES', 'LISTED_BUI', 'LISTED_PAR', 'P2_CLASS',
       'P3_CLASS', 'P3_Status', 'TITHE', 'GlobalID_1', 'EARLIEST_E', 'RESERVE',
       'FORESTRY', 'SURVEY_PRI', 'ACTION', 'SITE_ID', 'AreaHa', 'NAME',
       'Shape_Leng', 'OBJECTID_1', 'HE_REF', 'geometry']]
#Find intersections where old AWI is in same place as new AWI
gdf_new_reduced['intersects_old_AWI'] = gdf_new_reduced.geometry.apply(lambda x: gdf_old.geometry.intersects(x).any())


C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [157]:
#df = pd.read_excel('O:/NaturalEngland/Ancient Woodland Inventory Update/Analysis/AWI_data_analysis_assessment.xlsm', sheet_name = 'AWI_data_RAW')
df = gdf_new_reduced.copy()

In [158]:
# Defining functions
def check_if_assessed(FIELD):
    if FIELD == 3 or FIELD ==4 or FIELD ==5:
        return 0
    else:
        return 1

def check_if_present(FIELD):
    if FIELD == 1 or FIELD ==2 or FIELD ==6:
        return 1
    else:
        return 0
    
def check_if_zero(FIELD):
    if FIELD == 0:
        return 1
    else:
        return 0
    
def check_if_name(FIELD):
    if FIELD.strip() == '' or FIELD == 'None':
        return 0
    else:
        return 1


In [161]:
# Columns to check if if parcel has been marked as woodland
columns_to_assess = ['TITHE','C19_EM', 'OSD', 'CM_1773', 'C18_EM', 'C17_EM', 'EM','OTHER' ]

#Loop through each parcel and create new column showing if woodland present in source, and if source is zero (not woodland)
for i in columns_to_assess:
    df[f"present_{i}"] = df[i].apply(check_if_present)
    df[f"zero_{i}"] = df[i].apply(check_if_zero)
    df[f"assessed_{i}"] = df[i].apply(check_if_assessed)

df["assessed_HE_REF"] = df['HE_REF'].apply(check_if_name)

In [162]:
# Check which parcels are named

df['NAME_TITHE'] = df['NAME_TITHE'].astype(str)

names_to_assess = ['NAME_TITHE','NAME_EP1', 'NAME_OSD', 'NAME_EM', 'HE_REF']
# Add column showing which parcels are named in source
for i in names_to_assess:
    df[i] = df[i].astype(str)
    df[f"present_{i}"] = df[i].apply(check_if_name)

# Combine to get total named sources for parcel    
df['name_present'] = df['present_NAME_EM']+df['present_NAME_OSD']+df['present_NAME_EP1'] +df['present_NAME_TITHE']

In [164]:
# Combine columns showing woodland present to see how many sources show woodland present
df['present_combined'] = df['present_TITHE']+df['present_C19_EM'] +df['present_OSD']+df['present_CM_1773'] +df['present_C18_EM']+df['present_C17_EM'] +df['present_EM']+df['present_OTHER'] +df['present_HE_REF']
df['present_primary'] = df['present_TITHE'] + df['present_OSD'] +df['present_HE_REF']+df['present_CM_1773']
df['present_secondary'] =df['present_C19_EM'] +df['present_C18_EM'] +df['present_EM']+df['present_OTHER'] + df['present_C17_EM']

# Combine columns showing if zero present to show how many sources show no woodland present
df['zero_combined'] = df['zero_TITHE']+df['zero_C19_EM'] +df['zero_OSD']+df['zero_CM_1773'] +df['zero_C18_EM']+df['zero_C17_EM'] +df['zero_EM']+df['zero_OTHER'] 
df['zero_primary'] = df['zero_TITHE']+df['zero_OSD']+df['zero_CM_1773'] 
df['zero_secondary'] =  df['zero_C19_EM'] +df['zero_C18_EM']+df['zero_C17_EM'] +df['zero_EM']+df['zero_OTHER']

df['assessed_primary'] = df['assessed_TITHE'] + df['assessed_OSD'] +df['assessed_HE_REF']+df['assessed_CM_1773']


In [170]:
df[df.assessed_primary==4]

,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,TITHE_REF,C19_EM,C19_EM_REF,OSD,OSD_REF,CM_1773,C18_EM,C18_EM_REF,C17_EM,C17_EM_REF,EM,EM_REF,OTHER,OTHER_REF,NAME_TITHE,NAME_OSD,NAME_EM,TITHE_REF_,TITHE_LAND,TITHE_PLOT,TITHE_PL_1,COMMENTS,url,OSNAMES_UR,CHECK_NAME,TO_CHECK_N,NOTES,LISTED_BUI,LISTED_PAR,P2_CLASS,P3_CLASS,P3_Status,TITHE,GlobalID_1,EARLIEST_E,RESERVE,FORESTRY,SURVEY_PRI,ACTION,SITE_ID,AreaHa,NAME,Shape_Leng,OBJECTID_1,HE_REF,geometry,intersects_old_AWI,present_TITHE,zero_TITHE,assessed_TITHE,present_C19_EM,zero_C19_EM,assessed_C19_EM,present_OSD,zero_OSD,assessed_OSD,present_CM_1773,zero_CM_1773,assessed_CM_1773,present_C18_EM,zero_C18_EM,assessed_C18_EM,present_C17_EM,zero_C17_EM,assessed_C17_EM,present_EM,zero_EM,assessed_EM,present_OTHER,zero_OTHER,assessed_OTHER,present_NAME_TITHE,present_NAME_EP1,present_NAME_OSD,present_NAME_EM,present_HE_REF,name_present,assessed_HE_REF,present_combined,present_primary,present_secondary,zero_combined,zero_primary,zero_secondary,assessed_primary
1,None,None,WiltsAWI_01964_01,1,1,5,6,5,None,Preshute,5,None,2,None,0,5,None,5,None,5,None,5,None,Granham Down,None,None,NO,-,274,3,Tithe: too narrow to be WP,None,None,1,1,None,0,0,POT,R,WOOD,0,{162AE381-650D-48DA-80FC-99EDAEFF8F60},C19-Late,None,None,0,0,CWS_SU16.051,1.109747,Granham Down,994.517300,627,1300/46bH,"POLYGON ((418107.02 167993.37, 418098.12 16799...",False,0,1,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,2,2,0,2,2,0,4
5,Savernake Forest,None,WiltsAWI_02622_01,1,1,5,1,5,None,NA,5,None,2,None,2,0,EstateMap_Savernake_1300-360H,5,None,5,EstateMap_Savernake_1300-360H,5,None,None,None,None,NA,None,None,None,None,None,http://data.ordnancesurvey.co.uk/id/4000000074...,1,1,None,0,0,POT,R,WOOD,1,{5DB83524-BD06-4E0E-A52F-731851B2F1CA},C19-Late,None,None,0,0,SITE_0958,0.263408,Savernake Forest,336.195698,830,1300/360,"POLYGON ((423350.056 164337.362, 423345.413 16...",False,1,0,1,0,0,0,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,4,4,0,1,0,1,4
14,None,None,WiltsAWI_02000_01,0,1,5,2,5,None,Savernake,5,None,0,None,6,5,None,5,None,5,None,5,None,Brick Kiln Mead,None,None,NO,Pasture,117,3,None,None,None,0,0,None,0,0,DQ,INF,INF,0,{04F37689-F77C-40FB-8211-B30E6408BDC1},C19-Late,None,None,0,0,SITE_0843,0.121618,Brick Kiln Mead,154.845226,637,1300/375L,"POLYGON ((419604.16 167846.71, 419603.02 16784...",True,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,2,2,0,2,2,0,4
20,None,None,WiltsAWI_04365_06,1,1,5,1,5,None,Warminster,5,None,0,None,0,5,None,5,None,5,None,5,None,Field & Road,None,None,NO,Road & Pasture,1738,3,None,None,None,0,1,None,0,0,POT,R,PAWS,0,{F91BDFFB-BCFB-4B92-84C2-4450D210091C},C19-Late,None,None,0,0,SITE_1718,0.981584,Field & Road,480.340928,976,1332/3/2/3/11H,"POLYGON ((385682.03 143615.41, 385660.17 14359...",False,0,1,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,0,3,3,0,4
23,None,None,WiltsAWI_04709_01,0,1,5,6,5,None,Maiden Bradley,5,None,1,None,0,5,None,5,None,5,None,5,None,Little Common,None,None,NO,Pasture,216,3,None,None,None,0,1,None,0,0,POT,INF,INF,0,{5D99D6ED-9CF8-4D2D-A2F9-19F6ECC65060},C19-Late,None,None,0,0,SITE_1829,0.197259,Little Common,232.032589,1067,2056/1/2H,"POLYGON ((378564.769 139259.389, 378469.57 139...",False,0,1,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,2,2,0,2,2,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6841,"Horse Grove, Wraxall Park, Cullimer's Wood",Wraxall Park / Horse Gorse,WiltsAWI_01021_01,0,1,5,1,1,ge,North Wraxall,1,EstateMap_NorthWraxall_135-21,2,Malmesbury,2,2,EstateMap_NorthWraxall_2203-23,5,None,5,None,5,None,Horse Grove Wood||Wraxall Park,None,259: Hor

# Decision tree


In [145]:
# Filter for primary source present only
df_primary = df[df.present_primary >=1]
df_primary_one = df[df.present_primary ==1]
df_primary_two = df[df.present_primary >=2]
df_no_primary = df[df.present_primary ==0]

## A - Ancient woodland
## Contains primary evidence and no zero's present
df_primary_no_zero = df_primary[df_primary.zero_combined ==0]
df_primary_no_zero['Category'] = 'Ancient woodland'
df_primary_no_zero['subCategory'] = 'A'
df_one_primary_no_zero = df_primary_one[df_primary_one.zero_combined ==0]

#Filter for primary source present, one zero present
df_primary_zero = df_primary[df_primary.zero_combined >=1]
df_primary_one_zero = df_primary[df_primary.zero_combined ==1]
df_one_primary_zero = df_primary_one[df_primary_one.zero_combined >=1]
df_one_primary_one_zero = df_primary_one[df_primary_one.zero_combined ==1]

## A - Not ancient woodland 
## Has primary evidence but contains more than one zero
df_primary_plural_zero = df_primary[df_primary.zero_combined >1]
df_primary_plural_zero['Category'] = 'Not ancient woodland'
df_primary_plural_zero['subCategory'] = 'A'
df_one_primary_plural_zero = df_primary_one[df_primary_one.zero_combined >1]

## B - Ancient woodland
## Has primary evidence, one zero present, but is a named woodland 
df_primary_one_zero_named = df_primary_one_zero[df_primary_one_zero.name_present >=1]
df_primary_one_zero_named['Category'] = 'Ancient woodland'
df_primary_one_zero_named['subCategory'] = 'B'
df_one_primary_one_zero_named = df_one_primary_one_zero[df_one_primary_one_zero.name_present >=1]

## A- Maybe ancient woodland
## Has primary evidence, one zero present and is not a named woodland. 
### Further research needed, what source is zero?
df_primary_one_zero_not_named = df_primary_one_zero[df_primary_one_zero.name_present==0]
df_primary_one_zero_not_named['Category'] = 'Maybe ancient woodland'
df_primary_one_zero_not_named['subCategory'] = 'A'


C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python

In [95]:
# Filter for no primary evidence present, but secondary evidence present
df_secondary = df[(df.present_primary ==0)&(df.present_secondary >=1)]
# Filter for no evidence present
df_no_evidence = df[(df.present_primary ==0)&(df.present_secondary ==0)]

## D - Maybe ancient woodland
## No evidence, but contains no zeros, so may just not have been covered by maps
# Contains lots of 3/4's (no map coverage), requires further investigation to see if any maps preset
df_no_evidence_no_zero = df_no_evidence[df_no_evidence.zero_combined ==0]
df_no_evidence_no_zero['Category'] = 'Maybe ancient woodland'
df_no_evidence_no_zero['subCategory'] = 'D'

## C - Not ancient woodland
## No evidence, and contains at least one zero, so assumed not woodland. 
## 1637, ~600 1 zero, ~1000 more than one zero
df_no_evidence_zero = df_no_evidence[df_no_evidence.zero_combined >= 1]
df_no_evidence_zero['Category'] = 'Not ancient woodland'
df_no_evidence_zero['subCategory'] = 'C'

df_no_evidence_one_zero = df_no_evidence[df_no_evidence.zero_combined == 1]
df_no_evidence_plural_zero = df_no_evidence[df_no_evidence.zero_combined > 1]



C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python

In [96]:
#Filter for secondary evidence present, and no zeros present
df_secondary_no_zero = df_secondary[df_secondary.zero_combined ==0]

## C - Ancient woodland 
## No primary evidence, but contains secondary evidence, no zero's present and is named. 
df_secondary_no_zero_named = df_secondary_no_zero[df_secondary_no_zero.name_present >=1]
df_secondary_no_zero_named['Category'] = 'Ancient woodland'
df_secondary_no_zero_named['subCategory'] = 'C'

## B- Maybe ancient woodland
## No primary evidence, but contains secondary evidence, no zero's present, but is not named
## Further research needed; why is not present on primary evidence? 
df_secondary_no_zero_not_named  = df_secondary_no_zero[df_secondary_no_zero.name_present ==0]
df_secondary_no_zero_not_named['Category'] = 'Maybe ancient woodland'
df_secondary_no_zero_not_named['subCategory'] = 'B'

#Filter for secondary evidence present, and one zero present
df_secondary_one_zero = df_secondary[df_secondary.zero_combined ==1]

## C - Maybe ancient woodland
## No primary evidence, secondary evidence, but one zero present, and named
## Further research; why no primary evidence? and what source is zero? What source is name? 
df_secondary_one_zero_named  = df_secondary_one_zero[df_secondary_one_zero.name_present >=1]
df_secondary_one_zero_named['Category'] = 'Maybe ancient woodland'
df_secondary_one_zero_named['subCategory'] = 'C'

## E - Maybe ancient woodland
## No primary evidence, secondary evidence, but one zero present, not named
## Probably not ancient woodland, but check primary evidence, and zero. 
df_secondary_one_zero_not_named  = df_secondary_one_zero[df_secondary_one_zero.name_present ==0]
df_secondary_one_zero_not_named['Category'] = 'Maybe ancient woodland'
df_secondary_one_zero_not_named['subCategory'] = 'E'

## B - Not ancient woodland
## Multiple zeros present, and no primary evidence so probably not ancient woodland. 
df_secondary_plural_zero = df_secondary[df_secondary.zero_combined >1]
df_secondary_plural_zero['Category'] = 'Not ancient woodland'
df_secondary_plural_zero['subCategory'] = 'B'

df_secondary_zero = df_secondary[df_secondary.zero_combined >=1]

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\WSBRCvol\AppData\Local\Packages\PythonSoftwareFoundation.Python

In [97]:


df_combined = pd.concat([df_primary_no_zero,
                         df_primary_zero,
df_no_evidence_no_zero,
df_no_evidence_zero,
df_secondary_no_zero,
df_secondary_one_zero,
df_secondary_plural_zero,])

In [105]:
df_combined[df_combined.assessed_primary ==1]

,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,TITHE_REF,...,zero_primary,zero_secondary,assessed_primary,present_NAME_TITHE,present_NAME_EP1,present_NAME_OSD,present_NAME_EM,name_present,Category,subCategory
3,None,None,WiltsAWI_02310_01,1,1,5,1,5,None,Monkton Farleigh,...,0,0,1,1,0,0,0,1,Maybe ancient woodland,D
887,None,None,WiltsAWI_04050_01,0,1,5,6,5,None,NA,...,0,0,1,0,0,0,0,0,Maybe ancient woodland,D
1037,None,None,WiltsAWI_05015_01,0,1,5,1,5,None,Kilmington,...,0,0,1,1,0,0,0,1,Maybe ancient woodland,D


In [33]:
df_ancient_only = df_combined[df_combined.Category == 'Ancient woodland']
df_maybe_ancient = df_combined[df_combined.Category == 'Maybe ancient woodland']
df_not_ancient = df_combined[df_combined.Category == 'Not ancient woodland']

In [34]:
df_not_ancient_grouped = df_not_ancient.groupby('P3_Status').count().reset_index()
df_ancient_grouped = df_ancient_only.groupby('P3_Status').count().reset_index()

In [87]:
df_combined['area'] = df_combined.area/10000

In [98]:
df_combined

,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,TITHE_REF,...,zero_primary,zero_secondary,assessed_primary,present_NAME_TITHE,present_NAME_EP1,present_NAME_OSD,present_NAME_EM,name_present,Category,subCategory
9,None,None,WiltsAWI_05788_01,1,1,5,6,5,None,Wilton,...,0,0,3,1,0,0,0,1,Ancient woodland,A
12,None,None,WiltsAWI_01359_01,0,1,5,1,5,None,NA,...,0,0,3,0,0,0,0,0,Ancient woodland,A
19,None,None,WiltsAWI_04319_04,0,1,5,1,0,None,NA,...,0,0,3,0,0,0,1,1,Ancient woodland,A
21,None,None,WiltsAWI_04502_01,0,6,5,6,6,None,Horningsham,...,0,0,3,1,0,0,0,1,Ancient woodland,A
22,None,None,WiltsAWI_04628_01,0,6,5,6,5,None,Maiden Bradley,...,0,0,3,1,0,0,0,1,Ancient woodland,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5715,Hare Covert,None,WiltsAWI_04590_01,0,1,5,1,1,GX10135_SG_L_Exposure33,Codford St mary,...,2,0,3,1,0,0,0,1,Not ancient woodland,B
6594,Cowesfield Green,None,WiltsAWI_06689_01,0,1,5,1,1,GX10238_SG_L_Exposure66,Whiteparish,...,2,0,3,1,0,0,0,1,Not ancient woodland,B
6600,Cowesfield Green,None,WiltsAWI_06698_01,0,1,5,1,1,GX10238_SG_L_Exposure66,Whiteparish,...,2,0,3,1,0,0,0,1,Not ancient woodland,B
6607,Cowesfield Green,None,WiltsAWI_06713_01,0,1,5,1,1,GX10238_SG_L_Exposure66,Whiteparish,...,2,0,3,1,0,0,0,1,Not ancient woodland,B


In [174]:
#df_grouped = df_combined.groupby(['Category', 'subCategory','intersects_old_AWI']).count().reset_index()
df_grouped = df.groupby(['assessed_primary']).count().reset_index()


In [67]:
df_intersects_not_ancient = df_combined[(df_combined['intersects_old_AWI']==True)&(df_combined['Category']=='Not ancient woodland')]
df_intersects_not_ancient.area.sum()/10000

np.float64(670.9962464495912)

In [172]:
df_grouped

,assessed_primary,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,TITHE_REF,C19_EM,C19_EM_REF,OSD,OSD_REF,CM_1773,C18_EM,C18_EM_REF,C17_EM,C17_EM_REF,EM,EM_REF,OTHER,OTHER_REF,NAME_TITHE,NAME_OSD,NAME_EM,TITHE_REF_,TITHE_LAND,TITHE_PLOT,TITHE_PL_1,COMMENTS,url,OSNAMES_UR,CHECK_NAME,TO_CHECK_N,NOTES,LISTED_BUI,LISTED_PAR,P2_CLASS,P3_CLASS,P3_Status,TITHE,GlobalID_1,EARLIEST_E,RESERVE,FORESTRY,SURVEY_PRI,ACTION,SITE_ID,AreaHa,NAME,Shape_Leng,OBJECTID_1,HE_REF,geometry,intersects_old_AWI,present_TITHE,zero_TITHE,assessed_TITHE,present_C19_EM,zero_C19_EM,assessed_C19_EM,present_OSD,zero_OSD,assessed_OSD,present_CM_1773,zero_CM_1773,assessed_CM_1773,present_C18_EM,zero_C18_EM,assessed_C18_EM,present_C17_EM,zero_C17_EM,assessed_C17_EM,present_EM,zero_EM,assessed_EM,present_OTHER,zero_OTHER,assessed_OTHER,present_HE_REF,zero_HE_REF,assessed_HE_REF,present_combined,present_primary,present_secondary,zero_combined,zero_primary,zero_secondary,present_NAME_TITHE,present_NAME_EP1,present_NAME_OSD,present_NAME_EM,name_present,Category,subCategory
0,1,0,3,3,3,3,3,3,3,0,3,3,0,3,0,3,3,0,3,0,3,0,3,0,3,3,3,3,2,2,2,1,0,0,3,3,0,3,3,3,3,3,3,3,3,1,0,3,3,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,2,236,584,584,584,584,584,584,584,162,580,584,98,584,429,584,584,185,584,22,584,108,584,17,584,584,584,583,258,258,255,20,0,235,584,584,2,584,584,584,584,584,584,584,584,40,73,584,584,584,584,584,584,584,461,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,584,541,541
2,3,1705,4177,4177,4177,4177,4177,4177,4177,2131,4169,4177,359,4177,3621,4177,4177,410,4177,211,4177,193,4177,173,4177,4177,4177,4171,2883,2880,2851,233,26,1683,4177,4177,8,4177,4177,4175,4177,4174,4177,4177,4177,161,127,4177,4177,4177,4177,4177,4177,4177,690,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,4177,2763,2763
3,4,1129,2315,2314,2315,2315,2315,2315,2315,1168,2313,2315,151,2315,1981,2315,2315,132,2315,33,2315,41,2315,90,2315,2315,2315,2313,2248,2254,2213,132,35,1112,2315,2315,14,2315,2315,2313,2315,2313,2315,2315,2315,65,38,2315,2315,2313,2315,2314,2315,2315,394,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,2315,735,735


In [44]:
df_grouped_not_ancient  = df_grouped[df_grouped.Category =='Not ancient woodland']

In [175]:
fig = px.bar(df_grouped, x ='assessed_primary', y = 'P3_UID' , color='intersects_old_AWI')
fig.show()

# Decision tree diagram

In [146]:
df_one_primary_one_zero
df_sankey = {'source':[0,1,2,2,1,5,6,6,8,8, 5, 11,11,0, 14,15,15,17,17,14,21,21],
            'target':[ 1,2,3,4,5,6,7,8,9,10,11,12,13,21,15,16,17,18,19,20,14,22],
            'value':[df_no_primary.shape[0],df_no_evidence.shape[0],df_no_evidence_zero.shape[0],df_no_evidence_no_zero.shape[0],df_secondary.shape[0],df_secondary_zero.shape[0],
                    df_secondary_plural_zero.shape[0],df_secondary_one_zero.shape[0],df_secondary_one_zero_not_named.shape[0],df_secondary_one_zero_named.shape[0],
                    df_secondary_no_zero.shape[0],df_secondary_no_zero_not_named.shape[0],df_secondary_no_zero_named.shape[0],df_primary.shape[0],
                    df_one_primary_zero.shape[0],df_one_primary_plural_zero.shape[0],df_one_primary_one_zero.shape[0],df_primary_one_zero_not_named.shape[0],df_one_primary_one_zero_named.shape[0],
                    df_one_primary_no_zero.shape[0],df_primary_one.shape[0],df_primary_two.shape[0]],
            'colour':['#fb9985','#fb9985','#fb9985','#AFE1AF','#AFE1AF','#fb9985','#fb9985','#AFE1AF','#fb9985','#AFE1AF','#AFE1AF','#fb9985','#AFE1AF','#AFE1AF','#fb9985','#fb9985','#AFE1AF',
                '#fb9985','#AFE1AF','#AFE1AF','#fb9985','#AFE1AF'],
            'name':['primary source present?','secondary source present?','less than one zero present?','Contains zero','No zeros present','less than one zero present?','Only one zero?',
                'Multiple zeros present','Named?','Not named','Named','Named?','Not named','Named','less than one zero present?','Only one zero?','Multiple zeros present','Named?',
                'Not named','Named','No zeros present','Two primary sources present?','Two primary sources present'],
            'node_colour':['#ded6e2','#ded6e2','#ded6e2','#d0311a','#f8a54f','#ded6e2','#ded6e2','#d0311a','#ded6e2','#f8a54f','#f8a54f','#ded6e2','#f8a54f','#2ea411','#ded6e2','#ded6e2','#d0311a','#ded6e2',
                '#f8a54f','#2ea411','#2ea411','#ded6e2','#2ea411']}

In [155]:
df[df.P3_UID == 'WiltsAWI_04036_01']


,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,TITHE_REF,C19_EM,C19_EM_REF,OSD,OSD_REF,CM_1773,C18_EM,C18_EM_REF,C17_EM,C17_EM_REF,EM,EM_REF,OTHER,OTHER_REF,NAME_TITHE,NAME_OSD,NAME_EM,TITHE_REF_,TITHE_LAND,TITHE_PLOT,TITHE_PL_1,COMMENTS,url,OSNAMES_UR,CHECK_NAME,TO_CHECK_N,NOTES,LISTED_BUI,LISTED_PAR,P2_CLASS,P3_CLASS,P3_Status,TITHE,GlobalID_1,EARLIEST_E,RESERVE,FORESTRY,SURVEY_PRI,ACTION,SITE_ID,AreaHa,NAME,Shape_Leng,OBJECTID_1,HE_REF,geometry,intersects_old_AWI,present_TITHE,zero_TITHE,assessed_TITHE,present_C19_EM,zero_C19_EM,assessed_C19_EM,present_OSD,zero_OSD,assessed_OSD,present_CM_1773,zero_CM_1773,assessed_CM_1773,present_C18_EM,zero_C18_EM,assessed_C18_EM,present_C17_EM,zero_C17_EM,assessed_C17_EM,present_EM,zero_EM,assessed_EM,present_OTHER,zero_OTHER,assessed_OTHER,present_HE_REF,zero_HE_REF,assessed_HE_REF,present_combined,present_primary,present_secondary,zero_combined,zero_primary,zero_secondary,assessed_primary,present_NAME_TITHE,present_NAME_EP1,present_NAME_OSD,present_NAME_EM,name_present
365,None,None,WiltsAWI_04036_01,0,1,5,1,5,None,NA,5,None,2,Andover,4,5,None,5,None,5,None,5,None,None,None,None,NA,None,None,None,None,None,None,0,1,"Lots of trees have been removed, monitor that ...",0,0,POT,SPLIT,WOOD,4,{DE7B5785-4A26-4CFA-B9D7-9970079BB97E},C19-Early,None,None,0,0,SITE_1590,20.058221,WiltsAWI_04036_01,2470.880517,0,None,"POLYGON ((424233.4 147842.5, 424236 147846.65,...",True,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,2,0,0,0,0,0
388,None,None,WiltsAWI_04036_01,0,1,5,1,5,None,NA,5,None,2,Andover,4,5,None,5,None,5,None,5,None,None,None,None,NA,None,None,None,None,None,None,0,1,None,0,0,POT,SPLIT,PAWS,4,{84C3EBE4-4E98-44C8-9B0F-8033DC522DE2},C19-Early,None,None,0,0,SITE_1590,5.284747,WiltsAWI_04036_01,1054.356869,0,None,"POLYGON ((424629.34 148107.18, 424625.657 1481...",False,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,2,0,0,0,0,0


In [ ]:

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
     # line = dict(color = "black", width = 0.5),
      label =df_sankey['name'],
      color = df_sankey['node_colour']
    ),
    link = dict(
      source = df_sankey['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = df_sankey['target'],
      value = df_sankey['value'],
      color = df_sankey['colour']
  ))])

fig.update_layout(title_text="AWI Decision Tree", font_size=10)
fig.show()